# Algoritmo Genético

### Exercício 1

Proposta é modelar um algoritmo capaz de resolver um problema de solução conhecida. Neste caso, vamos iniciar de forma aleatória uma string e iterar, seguindo a lógica do algoritmo, até encontrar a solução desejada.

In [ ]:
import random
from datetime import datetime

def funcao_fitness(individuo):
    '''
    Nossa função fitness é simples, volta uma tupla contendo o valor esperado e
    o do indivíduo para cada letra da cadeia de caracteres
    Se a letra do indivíduo for igual a esperada, soma-se 1 ao resultado para
    este indivíduo
    '''
    return sum(1 for esperado, atual in zip(target, individuo) if esperado == atual)

def mostrar_resultado(individuo, fitness, geracao):
    deltaTempo = datetime.now() - inicioTempo
    print(f'Geração {geracao} -- {individuo}\t{fitness}\t{deltaTempo}')

def criar_individuo(tamanho):
    # Iniciamos uma lista para receber a sequência de caracteres
    genes = []
    '''
    Enquanto o tamanho da lista genes for menor que o tamanho recebido
    como parâmetro pela função, faça...
    '''
    while len(genes) < tamanho:
        '''
        Vamos popular a lista retirando 12 letras sem repetição do nosso
        geneSet
        '''
        tamanhoAmostra = min(tamanho - len(genes), len(geneSet))
        genes.extend(random.sample(geneSet, tamanhoAmostra))

    return ''.join(genes)

def mutacao(individuo):
    # Aleatoriamente escolhemos um número entre 0 e o tamanho do indivíduo
    index = random.randrange(0, len(individuo))
    # Retornamos 2 caracteres de nossa lista geneSet
    novoGene, alternar = random.sample(geneSet, 2)
    # Transformamos a cadeia de caracteres novamente em uma lista
    individuo = list(individuo)
    '''
    Se o indivíduo possuir em seu índice correspondente ao número aleatório
    que geramos acima o mesmo caractere que o obtido na variável novoGene,
    trocamos esse valor pelo caractere obtido na variável alternar. Caso
    contrário, substituímos esse índice pelo valor em novoGene
    '''
    individuo[index] = alternar if novoGene == individuo[index] else novoGene

    return ''.join(individuo)

random.seed(6)

'''
Primeiro iniciamos duas variáveis, uma com todas as possibilidades
de letras para resolver nosso problema e a segunda é a resposta
que esperamos encontrar com a busca
'''
geneSet = ' abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!.'
target  = 'Hello World!'

inicioTempo = datetime.now()
geracao = 1

'''
Iniciamos criando um indivíduo de maneira aleatória e calculamos
qual é o fitness obtido com este indivíduo...
Por ser o primeiro criado, adotamos que este é o melhor indivíduo
e o detentor do melhor fitness obtido
'''
melhor_individuo  = criar_individuo(len(target))
melhor_fitness    = funcao_fitness(melhor_individuo)

mostrar_resultado(melhor_individuo, melhor_fitness, geracao)

# Enquanto condição for verdade ou não faltar energia...
while True:

    geracao += 1

    # Cria um novo indivíduo aplicando mutação em seu pai
    filho = mutacao(melhor_individuo)

    # Calcula então qual o fitness obtido com este novo indivíduo
    filho_fitness = funcao_fitness(filho)

    mostrar_resultado(filho, filho_fitness, geracao)
    '''
    Se o valor do fitness do pai for maior ou igual ao do
    novo indivíduo, segue o loop sem processar o restante do bloco
    '''
    if melhor_fitness >= filho_fitness:
        continue
    '''
    Caso o fitness obtido seja maior ou igual ao tamanho da nossa variável
    de resultado, quer dizer que a as letras certas foram encontradas nas
    posições ideais, ou seja, temos o resultado esperado. Neste caso,
    quebro o bloco
    '''
    if filho_fitness >= len(target):
        break
    '''
    Se nenhum caso de comparação for satisfeito, então considero que o filho
    superou o pai, desta forma, ganha seu lugar
    '''
    melhor_individuo = filho
    melhor_fitness   = filho_fitness

### Exercício 2

Aqui veremos outra modelagem, porém mais robusta e considerando a criação de *n* indivíduos para resolver o mesmo problema proposto no **exercício 1**.

O objetivo é comparar os dois e gerar uma discussão sobre as duas estratégias e modelagens.

Fonte para o código original: [Github](https://gist.github.com/desertmonad/2299154)

In [ ]:
from collections import namedtuple
from operator import itemgetter
import string
import random


GA_POPSIZE = 1024    #ga population size
GA_MAXITER = 200     #maximum iterations
GA_ELITRATE = 0.2      #elitism rate (varname [sic])
GA_MUTATIONRATE = 0.25  
GA_MUTATION = 32768 * GA_MUTATIONRATE
GA_TARGET = "Hello World"

ga_struct = namedtuple("ga_struct", [ "string", "fitness"])


def init_population(ga_population, ga_buffer):
    tsize = len(GA_TARGET)

    for i in range(0,GA_POPSIZE):
        random_string = "".join((random.choice(string.ascii_letters+" ") for x in range(0,tsize)))
        citizen = ga_struct(random_string,0)
        
        ga_population.append(citizen)
        ga_buffer.append(ga_struct(" ",0))

def calc_fitness(ga_population):
    pop_index = 0
    for item in ga_population:
        index = 0
        fitness = 0
        for letter in item.string:
            fitness += abs(ord(letter) - ord(GA_TARGET[index]))
            index += 1
        ga_population[pop_index]  =  item._replace(fitness=fitness)
        pop_index += 1


def sort_by_fitness(ga_population):
    ga_population.sort(key=itemgetter(1))

def mutate(ga_population, index):
    rand_handler = random.Random()
    tsize = len(GA_TARGET)
    ipos = rand_handler.randint(0,tsize-1) - 1
    new_letter = random.choice(string.ascii_letters)
    citizen = ga_population[index]

    temp_list = list(citizen.string)
    temp_list[ipos]=new_letter
    citizen = citizen._replace(string="".join(temp_list))

    ga_population[index] = citizen


def elitism(ga_population, ga_buffer, esize):
    for i in range(0, esize):
        ga_buffer[i] = ga_population[i]
        
def mate(ga_population, ga_buffer):
    esize = int(GA_POPSIZE * GA_ELITRATE)
    tsize = len(GA_TARGET)
    rand_handler = random.Random()

    elitism(ga_population, ga_buffer, esize)

    #mate the rest
    for i in range(esize,GA_POPSIZE):
        i1 = rand_handler.randint(0,int(GA_POPSIZE/2))
        i2 = rand_handler.randint(0,int(GA_POPSIZE/2))
        spos = rand_handler.randint(0,tsize)
        
        ga_buffer[i] = ga_struct(ga_population[i1].string[0:spos] + ga_population[i2].string[spos:tsize],ga_buffer[i].fitness)
        if rand_handler.random() < GA_MUTATION:
           mutate(ga_buffer,i)
            
def main():
    my_population = []
    my_buffer = []

    init_population(my_population,my_buffer)

    last = 0
    for i in range(0,GA_MAXITER):
        calc_fitness(my_population)        
        sort_by_fitness(my_population)

        if my_population[0].fitness != last:
            print(" iter: %d,\t%s, \t score:%d " % (i, my_population[0].string, my_population[0].fitness))
            last = my_population[0].fitness

        if my_population[0].fitness == 0:
            break
        
        mate(my_population, my_buffer)
        (my_buffer, my_population) = (my_population, my_buffer) #swap

    print("done")

if __name__=="__main__":
    main()

### Exercício 3 - Proposta de leitura

Um problema muito conhecido por sua complexidade na computação é o do caixeiro viajante.

![title](http://www.carlospessoa.com.br/wp-content/uploads/2018/10/caixeiro_viajante.jpg)

Neste problema, o Rei solicita ao caixeiro que viaje por todo o reino anunciando o casamento de sua filha. O caixeiro então precisa definir qual é a menor rota para percorrer todos os vilarejos do reino.

![title](https://i0.wp.com/www.logisticadescomplicada.com/wp-content/uploads/2010/02/caixeiro-viajante-1.jpg?w=255&ssl=1)

![title](https://i1.wp.com/www.logisticadescomplicada.com/wp-content/uploads/2010/02/caixeiro-viajante-2.jpg?w=517&ssl=1)

Este problema é complexo, pois devemos calcular todas as possibilidades de caminhos entre os *n* vilarejos, dada sua complexidade, este calculo pode levar muito tempo. Um ótimo caso de solução utilizando algoritmos genéticos.

Veja o [artigo](https://towardsdatascience.com/evolution-of-a-salesman-a-complete-genetic-algorithm-tutorial-for-python-6fe5d2b3ca35) fonte do código anexado ao exercício, podemos encontrar uma descrição do problema e sua resolução.

In [ ]:
import numpy as np, random, operator, pandas as pd

class City:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def distance(self, city):
        xDis = abs(self.x - city.x)
        yDis = abs(self.y - city.y)
        distance = np.sqrt((xDis ** 2) + (yDis ** 2))
        return distance
    
    def __repr__(self):
        return "(" + str(self.x) + "," + str(self.y) + ")"

class Fitness:
    def __init__(self, route):
        self.route = route
        self.distance = 0
        self.fitness= 0.0
    
    def routeDistance(self):
        if self.distance ==0:
            pathDistance = 0
            for i in range(0, len(self.route)):
                fromCity = self.route[i]
                toCity = None
                if i + 1 < len(self.route):
                    toCity = self.route[i + 1]
                else:
                    toCity = self.route[0]
                pathDistance += fromCity.distance(toCity)
            self.distance = pathDistance
        return self.distance
    
    def routeFitness(self):
        if self.fitness == 0:
            self.fitness = 1 / float(self.routeDistance())
        return self.fitness

def createRoute(cityList):
    # Uma rota é criada, retirando 25 amostras sem reposição da lista de cidades
    route = random.sample(cityList, len(cityList))
    return route

def initialPopulation(popSize, cityList):
    population = []

    for i in range(0, popSize):
        population.append(createRoute(cityList))
    return population

def rankRoutes(population):
    fitnessResults = {}
    for i in range(0,len(population)):
        fitnessResults[i] = Fitness(population[i]).routeFitness()
    return sorted(fitnessResults.items(), key = operator.itemgetter(1), reverse = True)

def selection(popRanked, eliteSize):
    selectionResults = []
    df = pd.DataFrame(np.array(popRanked), columns=["Index","Fitness"])
    df['cum_sum'] = df.Fitness.cumsum()
    df['cum_perc'] = 100*df.cum_sum/df.Fitness.sum()
    
    for i in range(0, eliteSize):
        selectionResults.append(popRanked[i][0])
    for i in range(0, len(popRanked) - eliteSize):
        pick = 100*random.random()
        for i in range(0, len(popRanked)):
            if pick <= df.iat[i,3]:
                selectionResults.append(popRanked[i][0])
                break
    return selectionResults

def matingPool(population, selectionResults):
    matingpool = []
    for i in range(0, len(selectionResults)):
        index = selectionResults[i]
        matingpool.append(population[index])
    return matingpool

def breed(parent1, parent2):
    child = []
    childP1 = []
    childP2 = []
    
    geneA = int(random.random() * len(parent1))
    geneB = int(random.random() * len(parent1))
    
    startGene = min(geneA, geneB)
    endGene = max(geneA, geneB)

    for i in range(startGene, endGene):
        childP1.append(parent1[i])
        
    childP2 = [item for item in parent2 if item not in childP1]

    child = childP1 + childP2
    return child

def breedPopulation(matingpool, eliteSize):
    children = []
    length = len(matingpool) - eliteSize
    pool = random.sample(matingpool, len(matingpool))

    for i in range(0,eliteSize):
        children.append(matingpool[i])
    
    for i in range(0, length):
        child = breed(pool[i], pool[len(matingpool)-i-1])
        children.append(child)
    return children

def mutate(individual, mutationRate):
    for swapped in range(len(individual)):
        if(random.random() < mutationRate):
            swapWith = int(random.random() * len(individual))
            
            city1 = individual[swapped]
            city2 = individual[swapWith]
            
            individual[swapped] = city2
            individual[swapWith] = city1
    return individual

def mutatePopulation(population, mutationRate):
    mutatedPop = []
    
    for ind in range(0, len(population)):
        mutatedInd = mutate(population[ind], mutationRate)
        mutatedPop.append(mutatedInd)
    return mutatedPop

def nextGeneration(currentGen, eliteSize, mutationRate):
    popRanked = rankRoutes(currentGen)
    selectionResults = selection(popRanked, eliteSize)
    matingpool = matingPool(currentGen, selectionResults)
    children = breedPopulation(matingpool, eliteSize)
    nextGeneration = mutatePopulation(children, mutationRate)
    return nextGeneration

def geneticAlgorithm(population, popSize, eliteSize, mutationRate, generations):
    # Variável pop é criada
    pop = initialPopulation(popSize, population)
    print("Initial distance: " + str(1 / rankRoutes(pop)[0][1]))
    
    for i in range(0, generations):
        pop = nextGeneration(pop, eliteSize, mutationRate)
        print(f'Geração {i} - {1 / rankRoutes(pop)[0][1]}')
    
    print("Final distance: " + str(1 / rankRoutes(pop)[0][1]))
    bestRouteIndex = rankRoutes(pop)[0][0]
    bestRoute = pop[bestRouteIndex]
    return bestRoute




cityList = []
random.seed()

# De forma aleatória 25 cidades, com coordenadas (x, y) são criadas por exemplo:
# [(58,162), (87,42), (70,199), (126,147), (119,70), (142,69), (125,177), ...]
for i in range(0, 25):
    cityList.append(City(x=int(random.random() * 200), y=int(random.random() * 200)))

# Inicia o algoritmo genético
geneticAlgorithm(population=cityList, popSize=100, eliteSize=20, mutationRate=0.01, generations=500)